# Prep script

In [1]:
import time
import pylast
import hashlib
from datetime import date
from datetime import datetime
import requests
import pandas as pd, requests, time, json, os.path
import logging as lg, datetime as dt
import pandas as pd, numpy as np, matplotlib.pyplot as plt, logging as lg
import time, requests, re, json, os.path, datetime as dt
from geopy.distance import great_circle
import mpl_toolkits
import sys
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [8]:
%run -i 'Functions/all_recently_played_music.py'
%run -i 'Functions/get_locations.py'
%run -i 'Functions/geocode_locations.py'

<br>

# Set-up API call information

### Last.FM

In [4]:
# Enter your user details
user_name = 'pollystewart'
user_password = '77ebJ!134'
api_key = '040075c038c9f0e9200581b60723f860'
api_secret = '994d7d88b34019cb9c337fe179ac0f67'

# set-up hash / API call
password_hash = pylast.md5(user_password)
network = pylast.LastFMNetwork(api_key = api_key, api_secret =  api_secret, username = user_name, password_hash = password_hash)
user = pylast.User(user_name, network)

### Spotify

In [5]:
cid ="3fc91dc129ff4936beb0411d23003200" 
secret = "baa0c78d715f4497a8baedd3677516db"
sp = spotipy #spotipy.Spotify()

<br>

# Final things to do

 - Create a folder where this notebook is saved called 'raw_data' to save checkpoints
 - Create a folder where this notebook is saved called 'geocode' to save all geocode-related caches/jsons
 - Create a folder where this notebook is saved called 'place_Data' to save all geocode-related data files

<br>

# Get all music played

In [6]:
# just gets latest 5 as a check
recent_tracks = user.get_recent_tracks(limit = 5, cacheable=True)

In [ ]:
# gets max 999 tracks at a time
tracks = get_all_recent_tracks(user = user, network = network, limit_value = None)
tracks.head()

In [ ]:
tracks_w_tags = get_all_tags(tracks, network)

In [ ]:
tracks_w_tags

In [ ]:
tracks_w_tags.to_csv('data/raw_data/music_with_tags_20210807.csv', index = False)

<br>

# Get MBID of all artists/tracks/albums

In [ ]:
df_mbid = get_mbid_of_all_rows(tracks_w_tags, network)

In [ ]:
df_mbid.to_csv('data/raw_data/music_with_mbid_20210807.csv', index = False)

<br>

# Get Locations & MusicBrainz Info

In [ ]:
all_data = find_all_locations(df_mbid)

In [ ]:
all_data.to_csv("data/raw_data/music_with_places_20210807.csv", index = False)

<br>

# Get lat/lon coordinates

In [ ]:
music_with_lat_lon = get_all_latlons_for_music(all_data, ignore_country_if_more_detail = False)

In [ ]:
music_with_lat_lon.to_csv("data/raw_data/music_with_places_latlon_20210807.csv", index = False)

<br>

# Get info from Spotify

In [ ]:
spotify_data = get_spotipy_id(music_with_lat_lon, sp = sp, cid = cid, secret = secret)
spotify_data = spotify_data.drop_duplicates(["artist", "track"])

In [ ]:
spotify_data_info = get_spotify_data(spotify_data, batchsize_value = 100,  cid = cid, secret = secret)

In [ ]:
spotify_info = pd.merge(spotify_data, spotify_data_info, on = "track_id", how = "left", validate = "one_to_one" )

In [ ]:
spotify_info = spotify_info[['artist', 'track', 'popularity', 'acousticness', 'danceability', 'duration_ms', 
                             'energy','instrumentalness', 'liveness','loudness','mode','speechiness','tempo',
                             'time_signature', 'valence']]

In [ ]:
music_with_all_features = pd.merge(music_with_lat_lon, spotify_info, on = ["artist", "track"], how = "left", validate = "many_to_one" )


In [ ]:
music_with_all_features.to_csv("data/raw_data/music_with_all_features_20210807.csv", index = False)